In [ ]:
from cmdstanpy import CmdStanModel
import pandas as pd
import arviz as az
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from dags import plot_poisson_dag, plot_negative_binomial_dag
from functions import *
from sklearn.preprocessing import LabelEncoder

SEED = 18062025

In [ ]:
df = pd.read_csv("../globalterrorismdb_0718dist.csv", encoding='latin-1')

categorical_columns = ['weaptype1_txt', 'targtype1_txt', 'country_txt']
predictors = categorical_columns + ['nperps']

selected_columns = ['nkill'] + predictors
df = df[selected_columns].dropna()
print(len(df), "rows in the dataset before filtering")

df = df[df['nperps'] > 0 ]
df = df[df['nperps'] < 1000 ]
print(len(df), "rows in the dataset after filtering nperps > 0")

df = df.sample(n=2000, random_state=SEED)
print(len(df), "rows in the dataset after filtering")
df.head()


for column in categorical_columns:
    le = LabelEncoder()
    df[column.replace("txt", "encoded")] = le.fit_transform(df[column]) + 1

selected_columns = ["weaptype1_encoded", "targtype1_encoded", "country_encoded", "nperps"]
df_data = df[selected_columns]

weaptype1 = df['weaptype1_encoded'].values
targtype1 = df['targtype1_encoded'].values
country = df['country_encoded'].values
X_cont = df['nperps'].values
nkill = df['nkill'].values

stan_data = {
        'N': len(df),
        'K_cont': 1, # 1 since we got only 1 column
        'K_weaptype': int(weaptype1.max()),  # number of weapon types
        'K_targtype': int(targtype1.max()),  # number of target types
        'K_country': int(country.max()),     # number of countries
        'X_cont': X_cont.reshape(-1, 1),
        'weaptype': weaptype1.tolist(),
        'targtype': targtype1.tolist(),
        'country': country.tolist(),
        'nkill': np.array(nkill, dtype=int).tolist()
    }


model = CmdStanModel(stan_file='new_code.stan')
fit = model.sample(
    data=stan_data,
    seed=SEED,
    chains=4,
    iter_sampling=1000,
    iter_warmup=1000
)

In [ ]:
df_1_prod = fit.draws_pd()
hist_compare(df['nkill'], df_1_prod.iloc[0][8:])